## yaw

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf

modelh5 = tf.keras.models.load_model('/content/drive/MyDrive/light-frame-selection/Yaw_model.h5')

In [ ]:
import tensorflow as tf
import tf2onnx
import onnx



# Use from_function for tf functions
onnx_model, _ = tf2onnx.convert.from_keras(modelh5, opset=9)

In [ ]:
onnx.save(onnx_model, "model.onnx")

In [ ]:
import onnxruntime as ort
import numpy as np

xxx = np.zeros((1, 224, 224, 3), np.float32)


sess = ort.InferenceSession("/content/model.onnx")

# Set first argument of sess.run to None to use all model outputs in default order
# Input/output names are printed by the CLI and can be set with --rename-inputs and --rename-outputs
# If using the python API, names are determined from function arg names or TensorSpec names.


In [ ]:
import time

tt = time.time()
results_ort = sess.run(None, {"input_2": xxx})
time.time() - tt

0.4811058044433594

In [ ]:
max_anc = np.argmax(results_ort[2][0])
anchors = results_ort[1][0][max_anc]

if anchors[1] > 0:
    angle_offset = np.arccos(anchors[0])
else:
    angle_offset = -np.arccos(anchors[0])
bin_num = results_ort[2][0].shape[0]
wedge = 2. * np.pi / bin_num
theta_loc = angle_offset + max_anc * wedge

theta = theta_loc + 0 # theta_ray
    # object's yaw angle
yaw = (np.pi/2 - theta) * 57

if yaw < 0 :
    yaw = 360 + yaw

yaw += 90
if yaw > 360:
    yaw -= 360

yaw

180.524321694597

## glare

In [ ]:
import torch
import torchvision.models as models
import torch.nn as nn


num_classes = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.mobilenet_v2(weights="DEFAULT")

ii=1
for param in model.parameters():
    if ii < 133:
        param.requires_grad = False
    ii+=1


model.classifier[1] = nn.Sequential(
    nn.Dropout(p = 0.2),nn.Linear(1280, 500), nn.ReLU(),nn.Dropout(p = 0.5) ,  nn.Linear(500, num_classes)
)

model.to(device)
model.load_state_dict(torch.load('/content/best_model_epoch_9_cars.pth' , map_location=torch.device('cpu')))
model.to(device)
model.eval()
print("success!")

success!


In [ ]:
# Input to the model
import onnx

sz = 420
x = torch.randn(1, 3, sz, sz, requires_grad=True)
torch_out = model(x)

# Export the model
torch.onnx.export(model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "glare_9_cars.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=9,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [ ]:
import onnxruntime
import numpy as np

ort_session = onnxruntime.InferenceSession("glare_9_cars.onnx")


dict_keys(['input'])
Exported model has been tested with ONNXRuntime, and the result looks good!


In [ ]:
xxx = np.zeros((1, 3, 420, 420), np.float32)
ort_outs = ort_session.run(None, {"input":xxx})
ort_outs

[array([[-0.9553564,  1.0258516]], dtype=float32)]

# angle

In [ ]:
import torch
import torchvision.models as models
import torch.nn as nn


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.mobilenet_v2(weights="DEFAULT")

ii=1
for param in model.parameters():
    if ii < 0:
        param.requires_grad = False
    ii+=1


model.classifier[1] = nn.Sequential(
    nn.Dropout(p = 0.2),nn.Linear(1280, 500), nn.ReLU(),nn.Dropout(p = 0.5) ,  nn.Linear(500, 1)
)


model.to(device)
model.load_state_dict(torch.load('/content/angles_best_model_epoch_20.pth' , map_location=torch.device('cpu')))
model.to(device)
model.eval()
print("success!")

Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-7ebf99e0.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 69.9MB/s]


success!


In [ ]:
# Input to the model
import onnx

sz = 420
x = torch.randn(1, 3, sz, sz, requires_grad=True)
torch_out = model(x)

# Export the model
torch.onnx.export(model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "angle_mobileNet_20.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=9,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [ ]:
import onnxruntime
import numpy as np

ort_session = onnxruntime.InferenceSession("angle_mobileNet_20.onnx")

In [ ]:
xxx = np.zeros((1, 3, 420, 420), np.float32)
ort_outs = ort_session.run(None, {"input":xxx})
ort_outs

[array([[-8.878422]], dtype=float32)]

# blur
